<a href="https://colab.research.google.com/github/r42arty/hse/blob/main/term/red_remote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Бизнес-игра «Красная Удаленка»

## 🎯 Ваша цель — создать мультиагентную систему, способную эффективно управлять колонией на Марсе

## Начнём разработку вашей мультиагентной системы 👇
Следуйте по блокам ниже, начиная с Фазы 1.

📂 На 6-й день и далее вы загрузите файл `initiatives_day6_to_30.csv`, который выдаст ведущий.

In [ ]:
import pandas as pd
import random
import requests
import openai